In [36]:
"""
IMPROVING ON THE 50/150 MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500 AND 20 YEARS OF DATA

First,  we use the backtesting library to find a similar strategy to the conventional 50/150 strategy. We do this by 
using the backtesting library to test combinations of short SMAs between 40 and 60 with long SMAs between 130 and 170. 

Next, we confirm the results using our own backtesting and metrics.

The backtesting library choose short = 57, long =130 . It tested 800 combinations using short = 40 to 60 with a step of 1
and long = 130 to 170 with a step of 1. However, this did not beat a buy and hold strategy. 

A backtest showed a 146% return for the 57/130 strategy.

A backtest showed a 96% return using the 50/150 strategy. So we were able to improve upon the 50/150 strategy. 
 
However, both strategies underperformed the 206% return of a buy and hold strategy. 

"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

pd.set_option("display.max_rows", None, "display.max_columns", None)

%matplotlib inline

In [37]:
#load alpaca keys

load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)

str

In [38]:
#Use alpha vantage to download 20 years of data for the our selected ticker. 

ticker = 'SPY'

stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')
stock_data_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-06-29,301.4100,304.61,298.9300,304.46,304.46,73235506.0,0.0,1.0
2020-06-26,306.1600,306.39,299.4200,300.05,300.05,127811745.0,0.0,1.0
2020-06-25,303.4700,307.64,301.2800,307.35,307.35,88966079.0,0.0,1.0
2020-06-24,309.8400,310.51,302.1000,304.09,304.09,132067392.0,0.0,1.0
2020-06-23,313.4801,314.50,311.6101,312.05,312.05,68066900.0,0.0,1.0


In [39]:
#Clean data

#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2000-06-28,145.6250,146.9843,145.3125,145.5625,99.4832,5347700.0,0.0,1.0
2000-06-29,144.7500,145.7500,143.5156,144.1875,98.5435,6345700.0,0.0,1.0
2000-06-30,143.9375,145.5937,143.8906,145.2812,99.2910,7420200.0,0.0,1.0
2000-07-03,145.4375,147.4375,145.1875,147.2812,100.6579,1436600.0,0.0,1.0
2000-07-05,146.3750,146.6562,144.3750,144.6250,98.8425,2748200.0,0.0,1.0


In [40]:
#Set up stock data for backtesting, needs to be OHLCV

stock_data_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
stock_data_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
stock_data_df.sort_index(ascending=True, inplace=True)
stock_data_df.head()
#stock_data_df.to_csv('stock_data.csv')


,Open,High,Low,Close,Volume
date,,,,,
2000-06-28,145.6250,146.9843,145.3125,99.4832,5347700.0
2000-06-29,144.7500,145.7500,143.5156,98.5435,6345700.0
2000-06-30,143.9375,145.5937,143.8906,99.2910,7420200.0
2000-07-03,145.4375,147.4375,145.1875,100.6579,1436600.0
2000-07-05,146.3750,146.6562,144.3750,98.8425,2748200.0


In [12]:
#Function to do a backtest using Bactesting library. This function will find the best SMA cross strategy.

def backtest():
    class SmaCross(Strategy):
        
        n1=50
        n2=150
        
        def init(self):
            Close = self.data.Close
            self.ma1 = self.I(SMA, Close,self.n1)
            self.ma2 = self.I(SMA, Close,self.n2)

        def next(self):
            if crossover(self.ma1, self.ma2):
                self.buy()
            elif crossover(self.ma2, self.ma1):
                self.sell()

        
                
    bt_stock = Backtest(stock_data_df, SmaCross,
                              cash=10000, commission=.000)
  
    stats = bt_stock.optimize(n1=range(40, 60, 1),
                    n2=range(130, 170, 1),
                    maximize='Equity Final [$]',
                    constraint=lambda p: p.n1 < p.n2)
   

    bt_stock.run()
    #bt_spy_plot = bt_stock.plot()
    #return bt_stock_plot
    return stats
    #return bt_stock.run()
backtest()


C:\Users\costa\Anaconda3\envs\vizenv\lib\site-packages\ipykernel_launcher.py:28: UserWarning:

Searching best of 800 configurations.



Start                        2000-06-28 00:00:00
End                          2020-06-29 00:00:00
Duration                      7306 days 00:00:00
Exposure [%]                             93.4985
Equity Final [$]                         24619.9
Equity Peak [$]                          39830.6
Return [%]                               146.199
Buy & Hold Return [%]                    206.042
Max. Drawdown [%]                       -52.5541
Avg. Drawdown [%]                       -10.4175
Max. Drawdown Duration         974 days 00:00:00
Avg. Drawdown Duration          98 days 00:00:00
# Trades                                      33
Win Rate [%]                             54.5455
Best Trade [%]                            42.979
Worst Trade [%]                         -11.3787
Avg. Trade [%]                           4.74574
Max. Trade Duration           1097 days 00:00:00
Avg. Trade Duration            207 days 00:00:00
Expectancy [%]                           9.82757
SQN                 

In [35]:
#For comparison we run a backtest on the 50/150 strategy. This will only test the 50/150 strategy. 

class SmaCross(Strategy):
    def init(self):
        Close = self.data.Close
        self.ma1 = self.I(SMA, Close, 50)
        self.ma2 = self.I(SMA, Close, 150)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(stock_data_df, SmaCross,
              cash=10000, commission=0)
bt.run()

#The backtest shows a 96% return using the 50/150 strategy. 
#This is less than the 146% return using the 57/130 strategy. 
#Both strategies underperformed the 206% return of a buy and hold strategy. 







Start                     2000-06-28 00:00:00
End                       2020-06-29 00:00:00
Duration                   7306 days 00:00:00
Exposure [%]                           90.939
Equity Final [$]                        19608
Equity Peak [$]                       32485.2
Return [%]                            96.0803
Buy & Hold Return [%]                 206.042
Max. Drawdown [%]                    -56.7549
Avg. Drawdown [%]                    -8.71345
Max. Drawdown Duration     1588 days 00:00:00
Avg. Drawdown Duration      121 days 00:00:00
# Trades                                   29
Win Rate [%]                          51.7241
Best Trade [%]                        47.9952
Worst Trade [%]                       -12.467
Avg. Trade [%]                        4.67101
Max. Trade Duration        1113 days 00:00:00
Avg. Trade Duration         230 days 00:00:00
Expectancy [%]                        10.5969
SQN                                   1.15656
Sharpe Ratio                      

In [41]:
# To confirm the above results (short = 57, long = 130) we will do our own tests. 

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
date,
2000-06-28,99.4832
2000-06-29,98.5435
2000-06-30,99.2910
2000-07-03,100.6579
2000-07-05,98.8425


In [14]:
# Set the short window and long windows
short_window = 57
long_window = 130

# Set the `date` column as the index
#signals_df.set_index("date", drop=True)

# Generate the short and long moving averages (215 and 225 days, respectively)
signals_df["SMA57"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA130"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA57"][short_window:] > signals_df["SMA130"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA57,SMA130,Signal,Entry/Exit
date,,,,,
2000-06-28,99.4832,NaN,NaN,0.0,NaN
2000-06-29,98.5435,NaN,NaN,0.0,0.0
2000-06-30,99.2910,NaN,NaN,0.0,0.0
2000-07-03,100.6579,NaN,NaN,0.0,0.0
2000-07-05,98.8425,NaN,NaN,0.0,0.0
2000-07-06,99.6114,NaN,NaN,0.0,0.0
2000-07-07,101.2132,NaN,NaN,0.0,0.0
2000-07-10,101.0423,NaN,NaN,0.0,0.0
2000-07-11,101.2559,NaN,NaN,0.0,0.0


In [15]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA57', 'SMA130']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (Close)
   .Scatter.Close.II :Scatter   [date]   (Close)

In [16]:
# Set initial capital
initial_capital = float(100000)

signals_df["Portfolio Cash"] = 100000

# Set the share size
share_size = 1307

# Take a position where the dual moving average crossover is 1 (short SMA is greater than long SMA)
signals_df['Position'] = share_size * signals_df['Signal']

#Find the points in time where a share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = signals_df['Portfolio Cash'] - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# # Print the DataFrame
# pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.head(50)



,Close,SMA57,SMA130,Signal,Entry/Exit,Portfolio Cash,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2000-06-28,99.4832,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2000-06-29,98.5435,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,NaN,NaN
2000-06-30,99.2910,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2000-07-03,100.6579,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2000-07-05,98.8425,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2000-07-06,99.6114,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2000-07-07,101.2132,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2000-07-10,101.0423,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000
2000-07-11,101.2559,NaN,NaN,0.0,0.0,100000.0000,0.0,0.0,0.0000,100000.0000,0.000000,0.000000


In [18]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [19]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [20]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df



,Backtest
Annual Return,NaN
Cumulative Returns,2.08659
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [21]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0635587
Cumulative Returns,2.08659
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [22]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0635587
Cumulative Returns,2.08659
Annual Volatility,0.118706
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [23]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.0635587
Cumulative Returns,2.08659
Annual Volatility,0.118706
Sharpe Ratio,0.535428
Sortino Ratio,NaN


In [24]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

,Backtest
Annual Return,0.0635587
Cumulative Returns,2.08659
Annual Volatility,0.118706
Sharpe Ratio,0.535428
Sortino Ratio,NaN


In [25]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,0.0635587
Cumulative Returns,2.08659
Annual Volatility,0.118706
Sharpe Ratio,0.535428
Sortino Ratio,0.730115


In [26]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [27]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,2001-07-16,2001-08-14,1307.0,83.4271,82.4594,109039.2197,107774.4358,-1264.7839
1,SPY,2002-01-11,2002-04-24,1307.0,79.9917,76.3787,104549.1519,99826.9609,-4722.1910
2,SPY,2002-12-27,2003-02-25,1307.0,61.7946,59.7367,80765.5422,78075.8669,-2689.6753
3,SPY,2003-05-20,2004-06-23,1307.0,65.6455,83.0650,85798.6685,108565.9550,22767.2865
4,SPY,2004-10-27,2005-05-10,1307.0,82.0499,85.7451,107239.2193,112068.8457,4829.6264
5,SPY,2005-07-07,2006-07-05,1307.0,88.5631,95.5176,115751.9717,124841.5032,9089.5315
6,SPY,2006-09-18,2007-09-26,1307.0,99.7647,117.0498,130392.4629,152984.0886,22591.6257
7,SPY,2007-11-01,2008-01-02,1307.0,116.1576,112.0496,151817.9832,146448.8272,-5369.1560
8,SPY,2008-06-11,2008-07-22,1307.0,104.0562,99.5412,136001.4534,130100.3484,-5901.1050
9,SPY,2009-05-27,2010-07-02,1307.0,71.5001,83.5194,93450.6307,109159.8558,15709.2251


In [29]:
#Create plots

price_df = signals_df[['Close', 'SMA57', 'SMA130']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [30]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

In [32]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [33]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [34]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)